In [49]:
import os.path

import numpy as np
import pandas as pd

In [50]:
df = pd.read_csv("/Users/balazsmorvay/PycharmProjects/VAE/Data/usables.csv", sep=';')
for i in range(len(df)):
    filename = df.loc[i, 'FILE_ID']
    filepath = os.path.join('/Users/balazsmorvay/Downloads/ABIDE/data/Outputs/ccs/filt_noglobal/func_preproc',
                            f'{filename}_func_preproc.nii.gz')
    df.loc[i, "EXISTS"] = 1 if os.path.exists(filepath) else 0
df = df[df.EXISTS == 1]
df.head()

,FILE_ID,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,HANDEDNESS_CATEGORY,SITE_ID,EXISTS
1,Pitt_0050004,1,1,1,19.09,R,PITT,1
2,Pitt_0050005,1,1,2,13.73,R,PITT,1
3,Pitt_0050006,1,1,1,13.37,L,PITT,1
4,Pitt_0050007,1,1,1,17.78,R,PITT,1
5,Pitt_0050008,1,1,1,32.45,R,PITT,1


In [51]:
df['SITE_ID'].value_counts()

SITE_ID
NYU         169
UM_1         81
USM          60
UCLA_1       55
YALE         47
TRINITY      44
MAX_MUN      42
PITT         41
KKI          39
STANFORD     33
SDSU         31
UM_2         31
LEUVEN_1     28
LEUVEN_2     26
OLIN         25
SBL          23
OHSU         22
UCLA_2       20
CALTECH      15
CMU           2
Name: count, dtype: int64

In [52]:
df = df[df.SITE_ID == 'NYU']
timesteps = np.arange(0, 175)
df = df.merge(pd.Series(timesteps, name='TIME_SLICE'), how='cross')
train_df = df.sample(frac=0.8,random_state=42)
remaining = df.drop(train_df.index)
val_df = remaining.sample(frac=0.5, random_state=42)
test_df = remaining.drop(val_df.index)

In [53]:
train_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,23660.000000,23660.000000,23660.000000,23660.000000,23660.0,23660.000000
mean,1.571682,0.583178,1.208749,15.389336,1.0,86.964455
std,0.494845,0.774212,0.406423,6.613007,0.0,50.485709
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.520000,1.0,43.000000
50%,2.000000,0.000000,1.000000,13.710000,1.0,87.000000
75%,2.000000,1.000000,1.000000,19.640000,1.0,131.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [54]:
val_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,2958.000000,2958.000000,2958.000000,2958.000000,2958.0,2958.000000
mean,1.578431,0.573360,1.202840,15.447469,1.0,86.748479
std,0.493894,0.774461,0.402182,6.546772,0.0,50.400657
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.650000,1.0,42.000000
50%,2.000000,0.000000,1.000000,13.720000,1.0,88.000000
75%,2.000000,1.000000,1.000000,19.730000,1.0,130.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [55]:
test_df.describe()

,DX_GROUP,DSM_IV_TR,SEX,AGE_AT_SCAN,EXISTS,TIME_SLICE
count,2957.000000,2957.000000,2957.000000,2957.000000,2957.0,2957.000000
mean,1.587758,0.560027,1.198174,15.375878,1.0,87.536016
std,0.492322,0.769117,0.398691,6.560503,0.0,50.907407
min,1.000000,0.000000,1.000000,6.470000,1.0,0.000000
25%,1.000000,0.000000,1.000000,10.520000,1.0,43.000000
50%,2.000000,0.000000,1.000000,13.710000,1.0,89.000000
75%,2.000000,1.000000,1.000000,19.640000,1.0,132.000000
max,2.000000,3.000000,2.000000,39.100000,1.0,174.000000


In [56]:
train_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/train.csv')
val_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/val.csv')
test_df.to_csv('/Users/balazsmorvay/PycharmProjects/VAE/Configurations/ABIDE/test.csv')